자연어 처리 분약, 즉 텍스트의 경우에도 단어들이 모여서 문장이 되고, 문장이 모여 문서가 됩니다. 문장 내 단어들은 앞뒤 위치에 따라 서로 영향을 주고 받습니다. 문서 내 문장들도 마찬가지입니다. 따라서 순차적^{sequential}으로 입력을 넣고, 입력에 따라 모델의 $은닉\ 상태^{hidden\ state}$가 순차적으로 변하며, 상태에 따라 출력 결과가 순차적으로 반환되는 함수가 필요합니다.

이런 시간 개념 도는 순서 정보를 사용하여 입력을 학습하는 것을 **$시퀀셜\ 모델링^{sequential\ modelling}$**이라고 합니다. 신경망뿐 아니라 다양한 방법을 통해 이런 문제에 접근할 수 있습니다. 신경망에서는 **$순환\ 신경망^{RNN}$**이라는 아키텍쳐를 통해 효과적으로 문제를 해결할 수 있습니다.

### 순환 신경망

기존의 신경망 구조는 정해진 입력 x를 받아 y를 출력해주는 형태였습니다.

<br></br>
$$y = f(x;\theta)$$
<br></br>

이를 그림으로 표현하면 다음과 같습니다.

<br></br>
![](./images/7-2-1-nn.jpg)
<br></br>

하지만 순환 신경망은 입력 $x_t$와 직전의 은닉 상태인 $h_{t-1}$을 참조하여 현재의 상태인 $h_t$를 결정하는 작업을 여러 time-step에 걸쳐 수행합니다. 각 time-step별 RNN의 은닉 상태는 겨웅에 따라 출력값이 될 수 있습니다.

<br></br>
![](./images/7-2-1-rnn.jpg)
<br></br>

다음 수식처럼 RNN은 이전 time-step의 은닉 상태 $h_{t-1}$를 현재 time-step의 입력 $x_t$와 함께 받아 현재 time-step의 은닉 상태 $h_t$를 반환합니다.

<br></br>
$$h_t = f(x_t, h_{t-1};\theta)$$
<br></br>

### 피드포워드: 값이 앞으로 전달되는 과정

기본적은 RNN을 활용한 피드포워드 계산의 흐름을 살펴봅니다. 다음 그림은 각 time-step별로 입력 $x_t$와 이전 time-step의 $h_{t-1}$가 RNN으로 들어가서 출력 $h_t$를 반환하는 모습입니다. 이렇게 얻어낸 $h_t$들을 $\hat{y_t}$로 삼아서 정답인 $y_t$와 비교하여 손실 L을 계산합니다.

<br></br>
![](./images/7-2-1-ffn.jpg)
<br></br>

이 그림을 수식으로 표현하면 다음과 같습니다.

함수 f는 $x_t$와 $h_{t-1}$을 입력으로 받아서 파라미터 $\theta$를 통해 $h_t$를 계산합니다. 이때 각 입력과 출력, 그리고 내부 파라미터의 크기는 다음과 같습니다.

<br></br>
$$x_t \in R^w, h_t \in R^d, W_{ih} \in R^{dxw}, b \in R^d, W_{hh} \in R^{dxd}, b_{hh} \in R^d$$
<br></br>

입력 $x_t$를 받아서 입력에 대한 가중치 $W_{ih}, b_{ih}$를 곱하고 더한 후, 함께 입력으로 받은 이전 time-step의 은닉 상태 $h_{t-1}$과 가중치 $W_{hh}, b_{hh}$를 곱하고 더한 값을 모두 더해줍니다. 이후에 활성화 함수 `tanh`를 거쳐 현재 time-step의 은닉 상태 $h_t$를 반환합니다.

<br></br>
$$\hat{y} = h_t = f(x_t, h_{t-1}; \theta) \\
= tanh(W_{ih}x_t + b_{ih} + W_{hh}h_{t-1} + b_{hh}) \\
\text{where } \theta = \{W_{ih},b_{ih},W_{hh},b_{hh}\} $$
<br></br>

수식에서 나타나듯이 RNN에서는 RELU나 다른 활성화 함수들을 사용하기 보다는 tanh를 주로 사용합니다. 최종적으로 time-step별 $y_t$를 계산하여 다음 수식처럼 모든 time-step에 대한 손실 L을 구한 후, time-step의 수만큼 평균을 냅니다.

<br></br>
$$L = \frac{1}{n}\sum_{t=1}^{n}L(\hat{y_t},y_t)$$
<br></br>

#### RNN의 입력 텐서와 은닉 상태 텐서의 크기

입력으로 주어지는 $x_t$의 미니배치까지 감안한 크기는 다음과 같습니다.

<br></br>
$$x_t \in R^{batch\ size\ *\ 1\ *\ input\ size}$$
<br></br>

표현의 편의성을 위해 이 수식을 다음과 같이 표현하겠습니다.

<br></br>
$$|x_t| = (batch\ size, 1, input\ size)$$
<br></br>

텐서에서의 첫번째 차원, 배치 크기의 숫자의 의미는 미니 배치에서의 샘플 인덱스를 나타내며, 마지막 차원인 입력 크기는 미리 정해진 입력 벡터의 차원을 가리킵니다. 두번째 차원은 시퀀스 내에서 현재 time-step의 인덱스를 나타냅니다.

현재는 하나의 time-step에 대한 텐서이므로 1이 들어가있는 것을 알 수 있습니다. 그럼 n개의 time-step을 가진 전체 시퀀스를 텐서로 나타낸다면 다음 그림과 같습니다.

<br></br>
![](./images/7-2-1-input.jpg)
<br></br>

이것을 수식으로 나타내면 다음과 같습니다.

<br></br>
$$|x_{1:n}| = (batch\ size, n, input\ size) \\
\text{where}\ x_{1:n} = \{x_1, x_2, \dots, x_n\} $$
<br></br>

앞의 크기의 텐서를 입력으로 받아 RNN은 매 time-step마다 은닉 상태를 갱신합니다. t번째 time-step의 은닉 상태 텐서의 크기는 다음과 같습니다.

<br></br>
$$|h_t| = (batch\ size, hidden\ size)$$
<br></br>

이것을 n개 time-step에 대해서 이어붙이면 RNN의 전체 time-step에 대한 출력 텐서가 됩니다.

<br></br>
![](./images/7-2-1-timestep.jpg)
<br></br>

다음 수식에서 세미콜론 (;)은 concatenate 작업을 의미합니다. 이어붙이기를 time-step의 차원에 대해서 수행하면 전체 time-step에 대한 은닉 상태가 앞에서와 같이 만들어집니다.

$$|h_{1:n}| = (batch\ size, n, input\ size) \\
\text{where}\ h_{1:n} = [h_1; h_2; \dots; h_n] $$

### BPTT

이렇게 피드포워드된 이후 역전파는 어떻게 될까요?

각 time-step의 RNN에 사용된 파라미터 $\theta$는 모든 시간에 공유되어 사용됩니다. 따라서 앞서 구한 손실 L에 미분을 통해 역전파를 수행하게되면, 각 time-step별로 뒤로부터 $\theta$의 기울기^{gradient}가 구해지고, 이전 time-step의 기울기에 더해집니다. 즉 t가 0에 가까워질수록 RNN 파라미터 $\theta$의 기울기는 각 time-step별 기울기가 더해져 점점 커집니다.

<br></br>
$$\frac{\partial{L}}{\partial{\theta}}=\sum_t\frac{\partial{L(y_t,\hat{y_t})}}{\partial{\theta}}$$
<br></br>

다음 그림에서는 점점 짙어지는 붉은색 선을 통해 RNN 역전파의 속성을 나타냈습니다. 이 속성을 *시간 축에 대해서 수행되는 역전파 방법*이라는 뜻으로 **$BPTT^{backpropagation\ through\ time}$**이라고 합니다.

<br></br>
![](./images/7-2-2-bptt.jpg)
<br></br>

이런 RNN 역전파 속성으로 인해, RNN은 마치 time-step 수만큼 계층이 존재하는 것과 마찬가지인 상태가 됩니다. 따라서 time-step이 길어짐에 따라 매우 깊은 신경망과 유사하게 동작합니다.


### 기울기 손실

BPTT로 인해 RNN은 마치 역전파를 할때 time-step만큼의 계층이 있는 것과 비슷한 속성을 띱니다.그런데 앞의 RNN 수식을 보면 활성화 함수로 `tanh` 함수가 사용된 것을 알 수 있습니다. 

`tanh` 함수는 다음과 같은 형태를 가집니다.

<br></br>
$$tanh(x) = \frac{1-e^{-x}}{1+e^{-x}} \\
sigmoid(x) = \frac{1}{1+e^{-x}} \\
= 2tanh(2x)-1$$
<br></br>

다음 그림을 보면 tanh의 양 끝의 함수값이 점차 -1 또는 1에 근접해가는 것을 알 수 있습니다. 문제는 이렇게 되면  tanh의 양 끝의 기울기가 0에 가까워진다는 것입니다.

<br></br>
![](./images/7-2-1-tanh.jpg)
<br></br>

따라서 tanh 양 끝의 값을 반환하는 층의 경우 기울기가 0에 가까워지고, 그 다음으로 미분 값을 전ㄷ날받는 계층은 제대로 된 미분값을 전달받을 수 없게 됩니다. 게다가 다음의 도함수 그래프에서 볼 수 있듯이, tanh와 sigmoid의 도함수는 모두 기울기 값이 1보다 작거나 같으므로, 층을 거칠수록 기울기의 크기는 작아질수 밖에 없습니다.

<br></br>
![](./images/7-2-1-derivative.jpg)
<br></br>

이를 $기울기\ 소실^{vanishing\ gradient}$이라고 합니다. 따라서 RNN과 같이 time-step이 많거나, RNN이 아니더라도 여러 계층구조를 갖는 신경망인 $다층\ 퍼셉트론^{multilayer\ perceptron}$(MLP)의 경우에는 이러한 기울기 소실 문제가 쉽게 발생하는데, 이는 RELU와 레지듀얼 커넥션등의 등장으로 기울기 소실은 더 이상 큰 문제가 아닙니다.

### 여러 계층을 갖는 RNN

기본적으로 time-step별로 RNN이 동작하지만, 하나의 time-step 내에서 여러 층의 RNN을 쌓아올릴수도 있습니다. 그림에서 시간의 흐름이 왼쪽에서 오른쪽 방향이라면, 여러 층을 아래에서 위로 쌓아오릅니다. 여러개의 RNN층이 쌓여 하나의 RNN를 이룰때 가장 위층의 은닉 상태가 전체 RNN의 출력이 됩니다.

당연히 층별로 파라미터 $\theta$를 공유하지 않고 따로 가집니다. 보통은 각 층 사이에 드롭아웃을 끼워넣기도 합니다.

<br></br>
![](./images/7-2-2-stackedrnn.jpg)
<br></br>

하나의 층만 갖는 기존의 RNN의 경우에는 은닉 상태와 출력값이 같은 값이였지만, 여러 층이 쌓여 이루어진 RNN의 경우에는 각 time-step의 RNN 전체 출력값은 맨 위층의 은닉 상태가 됩니다. 따라서 RNN의 출력 텐서의 크기는 앞서 소개한 1개 층을 가진 RNN과 여전히 같음을 알 수 있습니다.

<br></br>
![](./images/7-2-2-output.jpg)
<br></br>

이를 수식으로 나타내면 다음과 같습니다.

<br></br>
$$|y_{1:n} = (batch size, n, hidden size)$$
<br></br>

대신에 여러층을 가진 RNN의 은닉 상태의 크기는 다음과 같습니다.

<br></br>
![](./images/7-2-2-hidden.jpg)
<br></br>

이를 수식으로 나타내면 다음과 같습니다.

<br></br>
$$|h_{1:n} = (\#\ layers, batch size, hidden size)$$
<br></br>


### 양방향 RNN

RNN의 방향에 관해 이야기할 차례입니다. 

앞에서 이야기한 RNN은 time-step t를 1에서부터 시작해서 마지막 time-step에 이를때까지 차례로 입력받아 진행했습니다. 하지만 **$양방향^{bi-directional}$** RNN을 사용하면 기존의 정방향에 역방향이 추가되어 마지막 time-step에서부터 역방향으로 입력받아 진행합니다.

양방향 RNN의 경우에도 당연히 정방향과 역방향의 파라미터 $\theta$는 공유되지 않습니다.

<br></br>
![](./images/7-2-5-bidir.jpg)
<br></br>

보통 여러 층의 양방향 RNN을 쌓게되면, 각 층마다 두 방향의 time-step별 은닉 상태 값을 이어붙여서 다음 층의 방향별 입력으로 사용됩니다. 경우에 따라서는 전체 RNN층 가운데 일부 층만 양방향 RNN층을 사용하기도 합니다. 따라서 양방향 RNN의 은닉 상태 텐서의 크기는 다음과 같습니다.

<br></br>
![](./images/7-2-5-hidden.jpg)
<br></br>

이를 수식으로 나타내면 다음과 같습니다.

<br></br>
$$|h_t| = (\#\ direction\ x\ \#\ layers, batch\ size, hidden\ size)$$
<br></br>

### 자연어 처리에 RNN를 적용한 사례

RNN의 입출력은 기본적으로 다음과 같이 분류할 수 있습니다.

<br></br>

|입력|출력|비고|
|---|---|---|
|다수|단일|many to one|
|다수|다수|many to many|
|단일|다수|one to many|

<br></br>

<br></br>
![](./images/7-2-6-type.jpg)
<br></br>

#### 하나의 출력을 사용할 경우

가장 쉬운 사용 예는 다음과 같이 마지막 time-step의 출력 값만 사용할 경우입니다.

<br></br>
![](./images/7-2-6-manytoone.jpg)
<br></br>

가장 흔한 예제는 감성 분석과 같은 텍스트 분류 예제입니다. 이때 단어의 개수만큼 입력이 RNN에 들어가고, 마지막 time-step의 결과값을 받아서 `softmax` 함수를 통해 해당 입력 텍스트의 클래스를 예측하는 확률 분포를 근사하도록 동작합니다.

<br></br>
$$softmax(x_i) = \frac{e^{x_i}}{\sum_{j=1}^{|C|}e^{x_j}}$$
<br></br>

각 time-step별 입력 단어인 $x_i$는 불연속적인 이산 분포로부터 샘플링된 샘플입니다. $x_t$는 원핫 벡터로 encode되고, 임베딩 계층을 거쳐 정해진 차원의 단어 임베딩 벡터인 덴스 벡터로 표현되어 RNN에 입력으로 주어집니다.

정답 또한 불연속적 값인 단어 또는 클래스가 될 것 입니다. 따라서 우리는 softmax 함수를 통해 멀티눌리 확률 분포를 표현합니다. 또한 원래의 정답도 원핫 벡터가 되어 교차 엔트로피 손실함수를 통해 softmax 결과값 벡터와 비교하여 손실값을 구하게 됩니다.

<br></br>
$$CrossEntropy(x_i) = -\sum_{i=1}^{|C|}y_i log \hat{y_i} \\
\text{where y and}\ \hat{y}\ \text{is probability distribution, such as}\ \hat{y} = P(y|x;\theta)
$$
<br></br>

이때 RNN은 모든 time-step에 대해 출력을 반환하지만, 나머지를 버리고 하나의 time-step에 대해서만 값을 취합니다. 따라서 신경망은 역전파를 통해서 해당 time-step의 출력값에 필요한 정보를 모으도록 훈련될 것입니다.

#### 모든 출력을 사용할 경우

많이 이용하는 방법은 모든 time-step의 출력값을 사용하는 것입니다. 이 방법은 언어 모델, 기계번역을 구현할때도 쓰이지만, 문장을 입력으로 주고, 단어별 형태소를 분류하는 문제처럼 여러 가지 방법으로 응용할 수 있습니다.

입력과 출력 모두 불연속적인 값이 될 것입니다. 따라서 각 time-step별로 불연속적 샘플인 원핫 벡터를 입력으로 받아 임베딩 계층을 거쳐 덴스 벡터를 만들어 RNN을 거치고, RNN은 time-step별로 결과물을 출력한 뒤, time-step별로 softmax 함수를 거쳐 이산 확률 분포의 형태로 만듭니다. 이후 불연속적인 원핫 벡터로 구성된 정답과 비교하여 손실을 구합니다.

<br></br>
![](./images/7-2-6-manytomany.jpg)
<br></br>

대부분의 경우 RNN은 여러 층과 양방향으로 구현할 수 있습니다. 하지만 앞의 그림에서와 같이 입출력이 서로 같은 데이터를 공유할 경우에는 양방향 RNN을 사용할 수 없습니다. 더 구체적으로 설명하자면, 이전 time-step의 출력값이 현재 time-step의 입력으로 쓰이는 모델 구조라면 양방향 RNN을 사용할 수 없습니다. 이렇게 이전 자신의 상태(출력)가 현재 자신의 상태를 결정하는 모델을 **$자기회귀\ 모델^{autoregressive\ model}$** (AR)이라고 합니다. 하지만 형태소 분석기처럼 출력이 다음 time-step에 입력에 영향을 끼치지 않는 경우에는 양방향 RNN을 사용할 수 있습니다.